# Quickstart torchbearer

This notebook will give a quick intro to training PyTorch models with torchbearer and how it differs from base PyTorch. 

We’ll need to load in some data and define a model, then we can train it with a standard PyTorch training loop and with torchbearer to see how it compares.

Before we do that, lets import all the packages we'll need:

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms

### Data
Now lets build our dataloader. We'll use the CIFAR10 dataset from torchvision and create training and testing dataloaders. 

In [2]:
BATCH_SIZE = 128

normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                 std=[1, 1, 1])

dataset = torchvision.datasets.CIFAR10(root='./data/cifar', train=True, download=True,
                                        transform=transforms.Compose([transforms.ToTensor(), normalize]))

traingen =  torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False) 

testset = torchvision.datasets.CIFAR10(root='./data/cifar', train=False, download=True,
                                       transform=transforms.Compose([transforms.ToTensor(), normalize]))
testgen = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


### PyTorch Model

Now lets build our model. Here is a 3 layer CNN with batchnorm, ReLU activations and a final linear layer to classify to CIFAR10s 10 classes. We also create the optimizer and define the criterion. 

In [3]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.convs = nn.Sequential(
            nn.Conv2d(3, 16, stride=2, kernel_size=3),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 32, stride=2, kernel_size=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, stride=2, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.classifier = nn.Linear(576, 10)

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, 576)
        return self.classifier(x)


model = SimpleModel().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

### PyTorch training loop

First lets have a look at a standard PyTorch training loop for this type of task. 

In [4]:
nEpoch = 2
for epoch in range(nEpoch):  # loop over the dataset multiple times
    trainloader = iter(traingen)
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = inputs.cuda(), labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss = 0.99 * running_loss + 0.01 * loss.item()
        print("Epoch: {}, Batch: {}, Running Loss: {}".format(epoch, i, running_loss), end='\r')

print('\n **** Finished Training **** \n')

correct = 0
total = 0
for data in testgen:
    images, labels = data
    labels = labels.cuda()

    outputs = model(images.cuda())
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
    
print('Accuracy of the model on the 10000 test images: %d %%' % (
    100*correct / total))

Epoch: 1, Batch: 390, Running Loss: 1.0885878478414863
 **** Finished Training **** 

Accuracy of the model on the 10000 test images: 61 %


We reset the model and optimizer 

In [5]:
model = SimpleModel()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Torchbearer Training 

We can see that the PyTorch loop takes quite a few lines. Instead, in torchbearer we can achieve the same result with just 4 lines. 

In [10]:
from torchbearer import Trial

torchbearer_trial = Trial(model, optimizer, criterion, metrics=['acc', 'loss']).to('cuda')
torchbearer_trial.with_generators(train_generator=traingen, val_generator=testgen)
_ = torchbearer_trial.run(2)

1/2(v): 100%|██████████| 79/79 [00:01<00:00, 64.37it/s, val_acc=0.658, val_acc_std=0.474, val_loss=0.98, val_loss_std=0.12]


This looks good, we get running accuracy and loss metrics for both training and validation passes and we get a tqdm display to view it. However this tqdm isn't very pretty, and there exists a tqdm version for running in notebooks. 

We can try this out by turning off the default printing, setting verbose=0 in the run, and explicitly passing a Tqdm callback (specifying to use tqdm_notebook) to handle the printing. 

In [8]:
from torchbearer.callbacks import Tqdm
from tqdm import tqdm_notebook
torchbearer_trial = Trial(model, optimizer, criterion, metrics=['acc', 'loss'], callbacks=[Tqdm(tqdm_notebook)]).to('cuda')
torchbearer_trial.with_generators(train_generator=traingen, val_generator=testgen)
_=torchbearer_trial.run(2, verbose=0)

You might see new lines between each tqdm bar, this is apparently a dependancy problem between jupyter and tqdm and will get fixed eventually (probably). 

You can go [here](<https://torchbearer.readthedocs.io/en/latest/examples/quickstart.html>) for a slightly different version of this guide and for other examples of using torchbearer. 